# Test Framework Demo

In [1]:
import h3
import folium
import numpy as np
import os
import random

from clusterfinder.clusterfinder import (
    DBSCANClusterFinder,
    DIANAClusterFinder
)
from pathfinder.algo import (
    OutwardSpiralPathFinder, 
    BayesianHexSearch
)
from utils.hex import (
    hex_to_binary, 
    octal_list_to_binary, 
    binary_to_octal_list, 
    binary_to_hex, 
    hex_to_array_index,
    array_index_to_hex,
    distance_between_2_hexas,
)
from utils.viz import (
    gradient_color,
    add_hex_to_map,
    create_gif
)
from experiment.test_framework import TestFramework

In [2]:
# HOTSPOTS = [
#     # SUTD
#     (1.3409627986555042, 103.96213688745193), 
#     (1.3410171559591544, 103.96280445720055), 
#     (1.3401776374681358, 103.96217615626067), 
#     (1.3402319947892152, 103.96269269212951),
#     (1.3408722031467406, 103.9634961923699),
    
#     # CCP
#     (1.3353773813885768, 103.96326877256507),
#     (1.3348659829813332, 103.96388261732363),
#     (1.3341322372547322, 103.96249479439123),
#     (1.3339009961318973, 103.96315756880448),
# ]

# Run and validate
RES=15
NUM_HOTSPOT = 200
NUM_CASUALTY = 10
STEPS = 500
CENTRE = (1.3392911509416838, 103.95958286190708)
MAP = folium.Map(location=CENTRE, zoom_start=16, tiles='cartodb positron', max_zoom=24)

def visualize_hotspots_and_casualties(map: folium.Map, hotspots, casualty_locations):
    """
    hotspots: list of tuple of (lat, lng)
    casualty_locations: set of hex_idx
    """
    casualty_locations = [h3.h3_to_geo(hex_idx) for hex_idx in casualty_locations]
    
    for hs in hotspots:
        folium.Marker(location=hs, icon=folium.Icon(color='red', icon='fire')).add_to(map)
    for casualty in casualty_locations:
        folium.Marker(location=casualty, icon=folium.Icon(color='orange', icon='exclamation-sign')).add_to(map)
    return map

def visualize_path(map: folium.Map, search_outputs, max_outputs=3):
    """
    Visualize up to max_outputs paths on a map.
    
    :param map_object: folium.Map object to which the paths will be added
    :param search_outputs: list of dictionaries with 'hex_idx' and 'step_count'
    :param max_outputs: maximum number of paths to visualize
    :return: folium.Map object with paths added
    """
    for idx, output in enumerate(search_outputs.values()):
        if idx >= max_outputs:
            break
        add_hex_to_map(output, map)
    return map

In [3]:
## Visualizaiton code

# map = test_object.main_map
# hotspots, casualty_locations = test_object.hotspots, test_object.casualty_locations
# all_search_outputs = test_object.all_search_outputs

# map = visualize_hotspots_and_casualties(map, hotspots, casualty_locations)
# map = visualize_path(map, all_search_outputs)
# display(map)

## Clustering Evaluation: DIANA v DBSCAN

### 1000 hotspots

In [4]:
NUM_HOTSPOT = 1000

print("\nDIANA")
test_object = TestFramework(name="DIANA", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DIANAClusterFinder, threshold=0.1)
test_object.run(STEPS, only_cluster=True, print_output=False)

print("\nDBSCAN")
test_object = TestFramework(name="DBSCAN", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DBSCANClusterFinder, max_gap=0.5, min_pts=1)
test_object.run(STEPS, only_cluster=True, print_output=False)

DIANA
Number of clusters 190

Average Evaluation Metrics:
Average Cluster Avg Dist: 25.44
Average Cluster Std Dist: 9.84
Average Path Coverage: NA
Average Angle Curvature: NA
Average Casualties Captured: NA
Average Casualties Count: NA
Average Minimum Time Captured: NA
Average False Negatives: NA
DBSCAN
Number of clusters 1

Average Evaluation Metrics:
Average Cluster Avg Dist: 380.84
Average Cluster Std Dist: 145.31
Average Path Coverage: NA
Average Angle Curvature: NA
Average Casualties Captured: NA
Average Casualties Count: NA
Average Minimum Time Captured: NA
Average False Negatives: NA


### 100 hotspots

In [6]:
NUM_HOTSPOT = 100

print("\nDIANA")
test_object = TestFramework(name="DIANA", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DIANAClusterFinder, threshold=0.1)
test_object.run(STEPS, only_cluster=True, print_output=False)

print("\nDBSCAN")
test_object = TestFramework(name="DBSCAN", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DBSCANClusterFinder, max_gap=0.5, min_pts=1)
test_object.run(STEPS, only_cluster=True, print_output=False)


DIANA
Number of clusters 61

Average Evaluation Metrics:
Average Cluster Avg Dist: 26.7
Average Cluster Std Dist: 10.13
Average Path Coverage: NA
Average Angle Curvature: NA
Average Casualties Captured: NA
Average Casualties Count: NA
Average Minimum Time Captured: NA
Average False Negatives: NA

DBSCAN
Number of clusters 1

Average Evaluation Metrics:
Average Cluster Avg Dist: 376.93
Average Cluster Std Dist: 150.24
Average Path Coverage: NA
Average Angle Curvature: NA
Average Casualties Captured: NA
Average Casualties Count: NA
Average Minimum Time Captured: NA
Average False Negatives: NA


### 10 hotspots

In [21]:
NUM_HOTSPOT = 10

print("\nDIANA")
test_object = TestFramework(name="DIANA", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DIANAClusterFinder, threshold=0.1)
test_object.run(STEPS, only_cluster=True, print_output=False)

print("\nDBSCAN")
test_object = TestFramework(name="DBSCAN", res=RES)
test_object.init_mission(MAP, CENTRE, NUM_HOTSPOT, NUM_CASUALTY)
test_object.register_cluster_finder(DBSCANClusterFinder, max_gap=0.5, min_pts=1)
test_object.run(STEPS, only_cluster=True, print_output=False)


DIANA
Number of clusters 8

Average Evaluation Metrics:
Average Cluster Avg Dist: 20.55
Average Cluster Std Dist: NA
Average Path Coverage: NA
Average Angle Curvature: NA
Average Casualties Captured: NA
Average Casualties Count: NA
Average Minimum Time Captured: NA
Average False Negatives: NA

DBSCAN
Number of clusters 1

Average Evaluation Metrics:
Average Cluster Avg Dist: 315.19
Average Cluster Std Dist: 94.18
Average Path Coverage: NA
Average Angle Curvature: NA
Average Casualties Captured: NA
Average Casualties Count: NA
Average Minimum Time Captured: NA
Average False Negatives: NA


## Create GIF

In [ ]:
# if STEPS >= 200:
#     print("This gna take forever")
# output_path = os.path.join(os.getcwd(), "Spiral_path.gif")
# create_gif(output_path, test_spiral.probability_map, spiral_output, test_spiral.casualty_locations, test_spiral.casualty_detected, dpi=50)
# print("Spiral_path.gif done")
# output_path = os.path.join(os.getcwd(), "Bayes_path.gif")
# create_gif(output_path, test_bayes.probability_map, bayes_output, test_bayes.casualty_locations, test_bayes.casualty_detected, dpi=50)
# print("Bayes_path.gif done")